### Char-RNN Trump-like text generation - TESTING

In [1]:
import sys
import numpy as np
import string

from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation
from keras.utils.data_utils import get_file
import io

N_GPU = 1 # you can experiment with more GPUs, it gets interesting with a high SEQUENCE_LEN
SEQUENCE_LEN = 10
# BATCH_SIZE = 512
BATCH_SIZE = 128
EPOCHS = 20
# HIDDEN_LAYERS_DIM = 512
HIDDEN_LAYERS_DIM = 100
# LAYER_COUNT = 4
LAYER_COUNT = 2
DROPOUT = 0.2

/Users/admin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# loading the text
path = get_file('laskdhalhdla.txt', origin='https://raw.githubusercontent.com/casassg/meme_puller/master/LangModel/cleaned_captions_4.txt')
with io.open(path, encoding='utf-8') as f:
    text_train = f.read().split()
print('corpus length:', len(text_train))

# generic vocabulary
characters = sorted(list(set(text_train)))

VOCABULARY_SIZE = len(characters)
characters_to_ix = {c:i for i,c in enumerate(characters)}
print("vocabulary len = %d" % VOCABULARY_SIZE)

155648/149610 [===============================] - 0s 0us/step
corpus length: 25010
vocabulary len = 5068


In [3]:
'STOP' in characters

True

In [4]:
test_model = Sequential()
for i in range(LAYER_COUNT):
    test_model.add(
            LSTM(
                HIDDEN_LAYERS_DIM, 
                return_sequences=True if (i!=(LAYER_COUNT-1)) else False,
                batch_input_shape=(1, 1, VOCABULARY_SIZE),
                stateful=True
            )
        )
test_model.add(Dense(VOCABULARY_SIZE))
test_model.add(Activation('softmax'))
test_model.compile(loss='categorical_crossentropy', optimizer="adam")

In [5]:
test_model.load_weights(
    "./1-gpu_BS-128_2-100_dp0.20_10S_epoch60-loss4.2825_weights"
)

In [6]:
def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array"""
    # from fchollet/keras
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def predict_next_char(model, current_char, diversity=1.0):
    """Predict the next character from the current one"""
    x = np.zeros((1, 1, VOCABULARY_SIZE))
    x[:,:,characters_to_ix[current_char]] = 1
    y = model.predict(x, batch_size=1)
    next_char_ix = sample(y[0,:], temperature=diversity)
    next_char = characters[next_char_ix]
    return next_char

def generate_text(model, seed="I", count=140):
    """Generate characters from a given seed"""
    model.reset_states()
    
    next_char = predict_next_char(model, seed)
    current_char = seed

    sys.stdout.write("["+seed+"]")
    
    for i in range(count - len(seed.split())):
        next_char = predict_next_char(model, current_char, diversity=1.0)
        if next_char == 'STOP':
            break
        current_char = next_char
        sys.stdout.write(' ')
        sys.stdout.write(next_char)
    print("\n")

In [8]:
for i in range(5):
    generate_text(
        test_model,
        seed="comic_book"
    )

[comic_book] level want days

[comic_book] rain april lul like games

[comic_book] oscar i trap

[comic_book] vegan the love comes the go

[comic_book] stupid what hard day

